In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
import random

model_version = 'lab_sm_v36'

folder_location = "" # you can put all these folders below in another folder if you want
wp_loc = "breaking_waves_lab_all"
non_wp_loc = "non_breaking_waves_lab_all"
wp_test_loc = "breaking_waves_lab_all_test"
non_wp_test_loc = "non_breaking_waves_lab_all_test"

"""%%%%%%%%%%%%%%%%%%%% GET IMAGES %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
# With the following settings, the model took about 30 minutes to train on my laptop
data = []
IMG_SIZE = 64
IMG_SIZE_WID = IMG_SIZE
brightness_change = 40
epochs = 20 # this was actually 60 for model 35, however that takes a while to train!
batch_size = 128 # this was actually 64 for model 35, however 64 doesn't always converge
training_val_ratio = 0.9
random.seed(1)

image_names = listdir(folder_location + wp_loc)
for i in range(0, len(image_names), 1):
    image_i = cv2.imread(folder_location + wp_loc + "/" + image_names[i])
    image_i_flipped = cv2.flip(image_i, 1)
    
    # get the image size to be normalized
    resized_image = cv2.resize(image_i, (IMG_SIZE_WID, IMG_SIZE))
    resized_image_flipped = cv2.resize(image_i_flipped, (IMG_SIZE_WID, IMG_SIZE))

    # increase the brightness of image
    resized_image_bi = resized_image.copy()
    resized_image_bi = np.where((255 - resized_image_bi) < brightness_change, 255, resized_image_bi + brightness_change)
    resized_image_bi_f = resized_image_flipped.copy()
    resized_image_bi_f = np.where((255 - resized_image_bi_f) < brightness_change, 255, resized_image_bi_f + brightness_change)

    # decrease the brightness of image
    resized_image_bd = resized_image.copy()
    resized_image_bd = np.where((resized_image_bd) < brightness_change, 0, resized_image_bd - brightness_change)
    resized_image_bd_f = resized_image_flipped.copy()
    resized_image_bd_f = np.where((resized_image_bd_f) < brightness_change, 0, resized_image_bd_f - brightness_change)

    # make a grayscale 3 channel image
    resized_image_gray = cv2.merge([cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY), 
                                    cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY), 
                                    cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)])
    resized_image_gray_f = cv2.merge([cv2.cvtColor(resized_image_flipped, cv2.COLOR_BGR2GRAY), 
                                      cv2.cvtColor(resized_image_flipped, cv2.COLOR_BGR2GRAY), 
                                      cv2.cvtColor(resized_image_flipped, cv2.COLOR_BGR2GRAY)])
    
    #cv2.imshow("test_a", resized_image_flipped)
    #cv2.imshow("test_b", resized_image_bi)
    #cv2.imshow("test_c", resized_image_bd)
    #cv2.imshow("rig", resized_image_gray)
    #cv2.imshow("rigf", resized_image_gray_f)
    #cv2.waitKey()
    
    data.append([resized_image, 1]) # 1 represents a breaking wave
    data.append([resized_image_flipped, 1])
    data.append([resized_image_bi, 1])
    data.append([resized_image_bd, 1])
    data.append([resized_image_bi_f, 1])
    data.append([resized_image_bd_f, 1])
    data.append([resized_image_gray, 1])
    data.append([resized_image_gray_f, 1])
    
    
image_names = listdir(folder_location + non_wp_loc)
for i in range(0, len(image_names), 1):
    image_i = cv2.imread(folder_location + non_wp_loc + "/" + image_names[i])
    image_i_flipped = cv2.flip(image_i, 1)
    
    # get the image size to be normalized
    resized_image = cv2.resize(image_i, (IMG_SIZE_WID, IMG_SIZE))
    resized_image_flipped = cv2.resize(image_i_flipped, (IMG_SIZE_WID, IMG_SIZE))

    # increase the brightness of image
    resized_image_bi = resized_image.copy()
    resized_image_bi = np.where((255 - resized_image_bi) < brightness_change, 255, resized_image_bi + brightness_change)
    resized_image_bi_f = resized_image_flipped.copy()
    resized_image_bi_f = np.where((255 - resized_image_bi_f) < brightness_change, 255, resized_image_bi_f + brightness_change)

    # decrease the brightness of image
    resized_image_bd = resized_image.copy()
    resized_image_bd = np.where((resized_image_bd) < brightness_change, 0, resized_image_bd - brightness_change)
    resized_image_bd_f = resized_image_flipped.copy()
    resized_image_bd_f = np.where((resized_image_bd_f) < brightness_change, 0, resized_image_bd_f - brightness_change)
    
    # make a grayscale 3 channel image
    resized_image_gray = cv2.merge([cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY), 
                                    cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY), 
                                    cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)])
    resized_image_gray_f = cv2.merge([cv2.cvtColor(resized_image_flipped, cv2.COLOR_BGR2GRAY), 
                                      cv2.cvtColor(resized_image_flipped, cv2.COLOR_BGR2GRAY), 
                                      cv2.cvtColor(resized_image_flipped, cv2.COLOR_BGR2GRAY)])
    
    #cv2.imshow("test_a", resized_image)
    #cv2.imshow("test_b", resized_image_bi)
    #cv2.imshow("test_c", resized_image_bd)
    #cv2.imshow("rig", resized_image_gray)
    #cv2.imshow("rigf", resized_image_gray_f)
    #cv2.waitKey()
    
    data.append([resized_image, 0]) # 0 represents a non-breaking wave
    data.append([resized_image_flipped, 0])
    data.append([resized_image_bi, 0])
    data.append([resized_image_bd, 0])
    data.append([resized_image_bi_f, 0])
    data.append([resized_image_bd_f, 0])
    data.append([resized_image_gray, 0])
    data.append([resized_image_gray_f, 0]) 

# Now randomize rows of data
random.shuffle(data)

# Now setup a training set and test set
data_train = data[0:int(training_val_ratio*len(data))]
data_test = data[int(training_val_ratio*len(data)):]

# Now split x data from y data
x_train = np.array(list(map(list, zip(*data_train)))[0])
y_train = np.array(list(map(list, zip(*data_train)))[1])

# needs to all be numpy arrays to work in tensorflow
x_test = np.array(list(map(list, zip(*data_test)))[0])
x_test_show = x_test
y_test = np.array(list(map(list, zip(*data_test)))[1])

# normalize input features (which are the pixel values)
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

# reshape for convolutional compatability
x_train = x_train.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)
x_test = x_test.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)

"""%%%%%%%%%%%%%%%%%%%% MACHINE LEARNING %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
# create the tensorflow model
model = Sequential()

# like VGGnet 16
model.add(Conv2D(32, (3,3), input_shape = x_train.shape[1:]))
model.add(Activation("relu"))
model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="SAME"))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="SAME"))

model.add(Conv2D(128, (3,3)))
model.add(Activation("relu"))
model.add(Conv2D(128, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="SAME"))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, validation_split=0.1, epochs=epochs)

"""%%%%%%%%%%%%%%%%%%%% EVALUATION %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""

# validation loss and validation accuracy should be close to training loss and accuracy (otherwise overfitting!)
train_loss, train_acc = model.evaluate(x_train, y_train)
val_loss, val_acc = model.evaluate(x_test, y_test)

model.save('wave_tracking_tf_' + model_version + '.model')

test_data = []
test_image_names = listdir(folder_location + wp_test_loc)
for i in range(0, len(test_image_names), 1):
    image_i = cv2.imread(folder_location + wp_test_loc + "/" + test_image_names[i])
    resized_image = cv2.resize(image_i, (IMG_SIZE_WID, IMG_SIZE))

    test_data.append([resized_image, 1])

test_image_names_non = listdir(folder_location + non_wp_test_loc)
for i in range(0, len(test_image_names_non), 1):
    image_i = cv2.imread(folder_location + non_wp_test_loc + "/" + test_image_names_non[i])
    resized_image = cv2.resize(image_i, (IMG_SIZE_WID, IMG_SIZE))

    test_data.append([resized_image, 0])

# needs to all be numpy arrays to work in tensorflow
x_tests = np.array(list(map(list, zip(*test_data)))[0])
x_test_shows = x_tests
y_tests = np.array(list(map(list, zip(*test_data)))[1])

# normalize input features (which are the pixel values)
x_tests = tf.keras.utils.normalize(x_tests, axis=1)

# reshape for convolutional compatability
x_tests = x_tests.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)

predictions = model.predict([x_tests])

cv2.namedWindow('test output', cv2.WINDOW_NORMAL)
cv2.resizeWindow('test output', 400,300) # just a nice visualisation resolution
cost_scores = []
print("showing misclassified images with error > 0.3")
j = 0
for prediction in predictions:
    test_image = cv2.resize(x_test_shows[j], (IMG_SIZE_WID, IMG_SIZE))
    if prediction - y_tests[j] > 0.3: # only show the worst images
        print("diff = " + str(prediction - y_tests[j]))
        cv2.imshow("test output", test_image)
        cv2.waitKey()

    cost_scores.append((prediction - y_tests[j]) ** 2)
    j = j + 1

MSE_test = np.mean(cost_scores)

cv2.destroyAllWindows()

print("Original number of labelled images = " + str(len(listdir(folder_location + wp_loc)) 
                                                    + len(listdir(folder_location + non_wp_loc))))
print("Number of images after augmentation = " + str(len(data)))   
print("training loss = " + str(train_loss) + ", training accuracy = " + str(train_acc))
print("test loss     = " + str(val_loss) + ", test accuracy     = " + str(val_acc))
print("total MSE for test images = " + str(MSE_test))

# save text file with all results
with open(model_version + ".txt", 'w') as txt_file:
    
    txt_file.write("training loss               = " + "{:.5f}".format(train_loss))
    txt_file.write("\ntest loss                 = " + "{:.5f}".format(val_loss))
    txt_file.write("\ntraining accuracy         = " + "{:.5f}".format(train_acc))
    txt_file.write("\ntest accuracy             = " + "{:.5f}".format(val_acc))
    txt_file.write("\ntotal MSE for test images = " + "{:.5f}".format(MSE_test))
    txt_file.write("\ntotal no. aug images      = " + "{:.5f}".format(len(data)))
    

The next cell is useful to re-run the results for a particular model without having to train a new one (also you can't train the same model twice since it is not deterministic).

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
import random

model_version = 'lab_sm_v35'

folder_location = "" # you can put all these folders below in another folder if you want
wp_loc = "breaking_waves_lab_all"
non_wp_loc = "non_breaking_waves_lab_all"
wp_test_loc = "breaking_waves_lab_all_test"
non_wp_test_loc = "non_breaking_waves_lab_all_test"

"""%%%%%%%%%%%%%%%%%%%% GET IMAGES %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
data = []
IMG_SIZE = 64
IMG_SIZE_WID = IMG_SIZE # can change this if needed

model = tf.keras.models.load_model('wave_tracking_tf_' + model_version + '.model')

test_data = []
test_image_names = listdir(folder_location + wp_test_loc)
for i in range(0, len(test_image_names), 1):
    image_i = cv2.imread(folder_location + wp_test_loc + "/" + test_image_names[i])
    resized_image = cv2.resize(image_i, (IMG_SIZE_WID, IMG_SIZE))

    test_data.append([resized_image, 1])

test_image_names_non = listdir(folder_location + non_wp_test_loc)
for i in range(0, len(test_image_names_non), 1):
    image_i = cv2.imread(folder_location + non_wp_test_loc + "/" + test_image_names_non[i])
    resized_image = cv2.resize(image_i, (IMG_SIZE_WID, IMG_SIZE))

    test_data.append([resized_image, 0])

# needs to all be numpy arrays to work in tensorflow
x_tests = np.array(list(map(list, zip(*test_data)))[0])
x_test_shows = x_tests
y_tests = np.array(list(map(list, zip(*test_data)))[1])

# normalize input features (which are the pixel values)
x_tests = tf.keras.utils.normalize(x_tests, axis=1)

# reshape for convolutional compatability
x_tests = x_tests.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)

predictions = model.predict([x_tests])

cv2.namedWindow('test output', cv2.WINDOW_NORMAL)
cv2.resizeWindow('test output', 400,300) # just a nice visualisation resolution
cost_scores = []
print("showing misclassified images with error > 0.3")
j = 0
for prediction in predictions:
    test_image = cv2.resize(x_test_shows[j], (IMG_SIZE_WID, IMG_SIZE))
    if prediction - y_tests[j] > 0.3: # only show the worst images
        print("diff = " + str(prediction - y_tests[j]))
        cv2.imshow("test output", test_image)
        cv2.waitKey()

    cost_scores.append((prediction - y_tests[j]) ** 2)
    j = j + 1

MSE_test = np.mean(cost_scores)

cv2.destroyAllWindows()

print("total MSE for test images = " + str(MSE_test))